In [1]:
# Import all the required libraries.

import csv
import bs4
import nltk
import string
import requests
import numpy as np
import pandas as pd
from textblob import TextBlob
from bs4 import BeautifulSoup
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
# input.xlsx is converted to input.csv and uploaded to dataframe.

df = pd.read_csv('input.csv')
df

,URL_ID,URL
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...
1,38.0,https://insights.blackcoffer.com/what-if-the-c...
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...
3,40.0,https://insights.blackcoffer.com/will-machine-...
4,41.0,https://insights.blackcoffer.com/will-ai-repla...
...,...,...
958,NaN,NaN
959,NaN,NaN
960,NaN,NaN
961,NaN,NaN


In [3]:
# Nan is dropped from dataframe

df = df.dropna()
len(df)

114

In [4]:
# URL present in the dataframe is copied to url_list in form of a list.

url_list = [url for url in df['URL']]

# The URL present in url_list are accessed by chrome and complete HTML code for said page is stored in text form and in a list 
# called text and according to the serial in which url was saved in url_list.

text = []
for url in url_list:
    text.append(requests.get(url,headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}))

In [5]:
# The unrequired part of HTML code present in text is removed using .content,'html.parser'.

for i in range(len(text)):
    text[i] = bs4.BeautifulSoup(text[i].content,'html.parser')

In [6]:
# The required paragraph is copied to list called articles by finding and specifying the class in HTML under which 
# the paragraph is present.

articles = []
for text in text:
    articles.append(text.find('div', attrs = {'class':'td-post-content'}))

In [7]:
# articles list is checked for Null value and their position.

for i in range(len(articles)):
    if(articles[i] == None):
        print(i)

7
20
107


In [8]:
# The 3 indexes at which null value were found are dropped from database to prevent error in further code.

df = df.drop([df.index[7], df.index[20],df.index[107]])

In [9]:
# The null values are dropped from articles list and copied to a new list called new_article.

new_article = []
for val in articles:
    if val != None :
        new_article.append(val)

In [10]:
# HTML code is completely removed by using ".text" and \n present after removing the code is replaced by space(' '). 
# The remaining data in new_article is the data required for sentiment analysis.

for i in range(len(new_article)):
    new_article[i]= new_article[i].text.replace('\n',' ')

In [11]:
# A new list called stop_words is formed and the stop words provided in form of multiple text files are added to stop_words 
# list.

stop_words = []

stopWordsFile1 = 'StopWords_Auditor.txt'
for stop_word in open(stopWordsFile1, 'r').readlines():
    stop_words.append(stop_word.rstrip())

stopWordsFile2 = 'StopWords_Currencies.txt'
for stop_word in open(stopWordsFile2, 'r').readlines():
    stop_words.append(stop_word.rstrip())

stopWordsFile3 = 'StopWords_Generic.txt'
for stop_word in open(stopWordsFile3, 'r').readlines():
    stop_words.append(stop_word.rstrip())

stopWordsFile4 = 'StopWords_GenericLong.txt'
for stop_word in open(stopWordsFile4, 'r').readlines():
    stop_words.append(stop_word.rstrip())

stopWordsFile5= 'StopWords_DatesandNumbers.txt'
for stop_word in open(stopWordsFile5, 'r').readlines():
    stop_words.append(stop_word.rstrip())

stopWordsFile6= 'StopWords_Geographic.txt'
for stop_word in open(stopWordsFile6, 'r').readlines():
    stop_words.append(stop_word.rstrip())
    
stopWordsFile7= 'StopWords_Names.txt'
for stop_word in open(stopWordsFile7, 'r').readlines():
    stop_words.append(stop_word.rstrip())

In [12]:
# It saves the different number of sentences present in differet paragraphs present in new_article list. 
# The number of sentences is further saved in a new list called sentences.  

sentences = []
for article in new_article:
    sentences.append(len(sent_tokenize(article)))

In [13]:
cleaned_articles = [' ']*len(new_article)

In [14]:
# Markings such as '?', '.',',' and '!' are replaced with space(' '). 

for i in range(len(new_article)):
    for w in stop_words:
        cleaned_articles[i]= new_article[i].replace('?',' ').replace('.',' ').replace(',',' ').replace('!',' ')

In [15]:
# A new list called words is created and number of words present in the differet paragraphs is present in new_article list
# is saved into the words list

words = []
for article in new_article:
    words.append(len(word_tokenize(article)))

In [16]:
# A new list called words_cleaned is used to store number of words from cleaned_articles.

words_cleaned = []
for article in cleaned_articles:
    words_cleaned.append(len(word_tokenize(article)))

In [17]:
# Positive words and negative words are copied from positive-words.txt and negative-words.txt respectively and 
# stored in new lists called positive_words and negative_words respectively. Further positive score and negative
# score are calculated by finding postive words and negative words in respectively in the cleaned_article list.

positive_words = []
negative_words = []

positiveWordsFile = 'positive-words.txt'
for positive_word in open(positiveWordsFile, 'r').readlines():
    positive_words.append(positive_word.rstrip())
    
positive_score = [0]*len(new_article)
for i in range(len(new_article)):
    for word in positive_words:
        for letter in cleaned_articles[i].lower().split(' '):
            if letter==word:
                positive_score[i]+=1

negativeWordsFile = 'negative-words.txt'
for negative_word in open(negativeWordsFile, 'r').readlines():
    negative_words.append(negative_word.rstrip())
    
negative_score = [0]*len(new_article)
for i in range(len(new_article)):
    for word in negative_words:
        for letter in cleaned_articles[i].lower().split(' '):
            if letter==word:
                negative_score[i]+=1

In [18]:
words_cleaned = np.array(words_cleaned)
sentences = np.array(sentences)

In [19]:
df['POSITIVE SCORE'] = positive_score
df['NEGATIVE SCORE'] = negative_score

In [20]:
df['POLARITY SCORE'] = (df['POSITIVE SCORE']-df['NEGATIVE SCORE'])/ ((df['POSITIVE SCORE'] +df['NEGATIVE SCORE']) + 0.000001)

In [21]:
df['SUBJECTIVITY SCORE'] = (df['POSITIVE SCORE'] + df['NEGATIVE SCORE'])/( (words_cleaned) + 0.000001)

In [22]:
df['AVG SENTENCE LENGTH'] = np.array(words)/np.array(sentences)

In [24]:
# Complex words and syllables are counted here for calculating percentage of complex words, fog index, complex word count 
# and syllables per word.

complex_words = []
syllables_counts = []
for article in new_article:
    syllables_count = 0
    d=article.split()
    ans=0
    for word in d:
        count=0
        for i in range(len(word)):
            if(word[i]=='a' or word[i]=='e' or word[i] =='i' or word[i] == 'o' or word[i] == 'u'):
                count+=1
            if(i==len(word)-2 and (word[i]=='e' and word[i+1]=='d')):
                count-=1
            if(i==len(word)-2 and (word[i]=='e' and word[i]=='s')):
                count-=1
        syllables_count+=count   
        if(count>2):
            ans+=1
    syllables_counts.append(syllables_count)
    complex_words.append(ans)

In [25]:
df['PERCENTAGE OF COMPLEX WORDS'] = np.array(complex_words)/np.array(words)

In [26]:
df['FOG INDEX'] = 0.4 * (df['AVG SENTENCE LENGTH'] + df['PERCENTAGE OF COMPLEX WORDS'])

In [27]:
df['AVG NUMBER OF WORDS PER SENTENCES'] = df['AVG SENTENCE LENGTH']

In [28]:
df['COMPLEX WORD COUNT'] = complex_words

In [29]:
df['WORD COUNT'] = words

In [30]:
df['SYLLABLE PER WORD'] = np.array(syllables_counts)/np.array(words)

In [31]:
# Total number of characters are calculated for every paragraph and stored in list called total_characters.

total_characters = []
for article in new_article:
    characters = 0
    for word in article.split():
        characters+=len(word)
    total_characters.append(characters) 

In [32]:
# Total number of personal noun are counted and stored in a list called personal_nouns.

personal_nouns = []
personal_noun =['I', 'we','my', 'ours','and' 'us', 'We','My', 'Ours','And' 'Us'] 
for article in new_article:
    ans=0
    for word in article:
        if word in personal_noun:
            ans+=1
    personal_nouns.append(ans)

In [33]:
df['PERSONAL PRONOUN'] = personal_nouns

In [34]:
df['AVG WORD LENGTH'] = np.array(total_characters)/np.array(words)

In [35]:
# Final dataframe is printed.

df

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCES,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUN,AVG WORD LENGTH
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...,76,34,0.381818,0.060011,26.129870,0.301193,10.572425,26.129870,606,2012,1.859841,39,5.160040
1,38.0,https://insights.blackcoffer.com/what-if-the-c...,71,38,0.302752,0.074863,20.525000,0.195493,8.288197,20.525000,321,1642,1.542631,37,4.302680
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...,74,38,0.321429,0.064479,22.552941,0.279082,9.132809,22.552941,535,1917,1.811163,31,4.912363
3,40.0,https://insights.blackcoffer.com/will-machine-...,84,28,0.500000,0.067146,19.284211,0.199782,7.793597,19.284211,366,1832,1.581332,74,4.428493
4,41.0,https://insights.blackcoffer.com/will-ai-repla...,69,28,0.422680,0.053151,25.050633,0.210207,10.104336,25.050633,416,1979,1.643759,36,4.612936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,146.0,https://insights.blackcoffer.com/blockchain-fo...,28,27,0.018182,0.058761,20.673469,0.237907,8.364551,20.673469,241,1013,1.714709,15,4.950642
110,147.0,https://insights.blackcoffer.com/the-future-of...,46,16,0.483871,0.037148,28.451613,0.235828,11.474976,28.451613,416,1764,1.641723,42,4.667234
111,148.0,https://insights.blackcoffer.com/big-data-anal...,32,46,-0.179487,0.065217,19.636364,0.265432,7.960718,19.636364,344,1296,1.711420,20,4.658179
112,149.0,https://insights.blackcoffer.com/business-anal...,36,4,0.800000,0.054201,27.275862,0.294564,11.028170,27.275862,233,791,1.895070,8,5.260430


In [36]:
# The dataframe prepared is stored in a CSV file called ASSIGNMENT SOLUTION.csv. 

df.to_csv('ASSIGNMENT SOLUTION.csv')